# Findings 

# Set up

In [139]:
import requests
import pandas as pd

In [140]:
import numpy as np

In [141]:
from datetime import datetime
from datetime import date

# Get and check the data

In [142]:
pwd

'/Users/monicacordero/Documents/IM'

In [143]:
## osha accident abstract
df2 = pd.read_csv("/Users/monicacordero/Documents/IM/osha_accident_abstract.csv")
df2.head(1)

,summary_nr,line_nr,abstract_text,load_dt
0,38083,1,An employee working for Asplundh Tree Expert C...,2019-07-22 00:19:26 EDT


In [144]:
# osha accident df.describe() me da mean, min
df= pd.read_csv("osha_accident.csv")
df.head(1)

,summary_nr,report_id,event_date,event_time,event_desc,event_keyword,const_end_use,build_stories,nonbuild_ht,project_cost,project_type,sic_list,fatality,state_flag,abstract_text,load_dt
0,9407,112600,1984-03-25 00:00:00,NaN,NONE,NaN,NaN,NaN,NaN,NaN,NaN,2621,X,NaN,NaN,2019-07-22 00:18:27 EDT


In [145]:
# number of rows and column types
df.shape

(142981, 16)

In [146]:
## Cheking Columns, Non-Null, type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142981 entries, 0 to 142980
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   summary_nr     142981 non-null  int64  
 1   report_id      142981 non-null  int64  
 2   event_date     142981 non-null  object 
 3   event_time     0 non-null       float64
 4   event_desc     142979 non-null  object 
 5   event_keyword  142713 non-null  object 
 6   const_end_use  30759 non-null   object 
 7   build_stories  21528 non-null   float64
 8   nonbuild_ht    15317 non-null   float64
 9   project_cost   25498 non-null   object 
 10  project_type   31130 non-null   object 
 11  sic_list       109089 non-null  object 
 12  fatality       64554 non-null   object 
 13  state_flag     0 non-null       float64
 14  abstract_text  0 non-null       float64
 15  load_dt        142981 non-null  object 
dtypes: float64(5), int64(2), object(9)
memory usage: 17.5+ MB


In [147]:
## checking and filter by event_date column 
df.sort_values(by = ["event_date"],ascending = False)
df["event_date"] = pd.to_datetime(df["event_date"])
df_filtered = df[df["event_date"]>"2016-1-1"]


In [166]:
df_filtered.sort_values(by=["event_date"]).head(2)


,summary_nr,report_id,event_date,event_time,event_desc,event_keyword,const_end_use,build_stories,nonbuild_ht,project_cost,project_type,sic_list,fatality,state_flag,abstract_text,load_dt,event_year,event_month
112699,220819726,521700,2016-01-02,NaN,EMPLOYEE IS CRUSHED WHEN A CONCRETE WALL FALLS,"COLLAPSE,CONCRETE,CRUSHED,WALL",C,1.0,15.0,A,B,NaN,X,NaN,NaN,2022-03-20 00:18:48 EDT,2016,1
112718,220820799,830600,2016-01-02,NaN,WORKER IS KILLED IN OFFICE BUILDING FIRE,"FIRE,OFFICE AREA,SMOKE,SMOKE INHALATION",NaN,NaN,NaN,NaN,NaN,NaN,X,NaN,NaN,2022-03-20 00:18:47 EDT,2016,1


In [149]:
# extract year and month

In [150]:
df_filtered["event_year"] = pd.to_datetime(df["event_date"]).dt.year
df_filtered["event_month"] = pd.to_datetime(df["event_date"]).dt.month

/Users/monicacordero/.pyenv/versions/3.8.10/lib/python3.8/site-packages/pandas-1.4.0.dev0+143.g5675cd8ab2-py3.8-macosx-11.3-arm64.egg/pandas/core/frame.py:3602: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/Users/monicacordero/.pyenv/versions/3.8.10/lib/python3.8/site-packages/pandas-1.4.0.dev0+143.g5675cd8ab2-py3.8-macosx-11.3-arm64.egg/pandas/core/frame.py:3602: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [157]:
df_filtered.shape

(29258, 18)

In [158]:
df_filtered.dtypes

summary_nr                int64
report_id                 int64
event_date       datetime64[ns]
event_time              float64
event_desc               object
event_keyword            object
const_end_use            object
build_stories           float64
nonbuild_ht             float64
project_cost             object
project_type             object
sic_list                 object
fatality                 object
state_flag              float64
abstract_text           float64
load_dt                  object
event_year                int64
event_month               int64
dtype: object

## Checking 

### Ten most common events since 2016

In [159]:
df_filtered["event_desc"].value_counts().head(10)

EMPLOYEE WITH COVID-19 INFECTION DIES               1232
EMPLOYEE WITH COVID-19 INFECTION IS HOSPITALIZED     146
EMPLOYEE FALLS FROM ROOF AND IS KILLED               110
EMPLOYEE IS KILLED IN FALL FROM ROOF                  94
EMPLOYEE FALLS FROM LADDER AND IS KILLED              90
EMPLOYEE DIES FROM HEART ATTACK                       66
EMPLOYEE IS KILLED IN FALL FROM LADDER                50
EMPLOYEE FALLS FROM SCAFFOLD AND IS KILLED            43
EMPLOYEE FALLS THROUGH SKYLIGHT AND IS KILLED         37
EMPLOYEE IS STRUCK AND KILLED BY MOTOR VEHICLE        29
Name: event_desc, dtype: int64

### Event per year

In [162]:
# df_filtered["event_desc"].value_counts().head(10)
df_filtered.groupby("event_year")["event_desc"].count()

event_year
2016    1757
2017    8839
2018    9086
2019    4967
2020    3143
2021    1465
2022       1
Name: event_desc, dtype: int64

In [165]:
## fact checking using summary_nr vs event_desc 
##### FALTA variacion 
df_filtered.groupby("event_year")["summary_nr"].count()

event_year
2016    1757
2017    8839
2018    9086
2019    4967
2020    3143
2021    1465
2022       1
Name: summary_nr, dtype: int64